# COVID-19 SNF Dashboard Update

Download data from https://www.cdph.ca.gov/Programs/CID/DCDC/Pages/COVID-19/SNFsCOVID_19.aspx

### IMPORTS

In [171]:
#import libraries
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import sys
import webbrowser

### SAVE DATA FROM THIS WEBSITE

1. Scroll down to Tableau dashboard
2. Check "report date" for data_date and file_date variables below
3. Select all San Diego County records in the table
4. Click the "Download" Button, then the "Data" button in the pop-up
5. Click "download all rows as a text file" then rename file to match naming convention 
6. Current naming convention is "raw_snf_covid_MMDDYY.csv"

In [168]:
# Open CDPH's COVID-19 SNF site in browser
# Save San Diego County records to CSV
webbrowser.open('https://www.cdph.ca.gov/Programs/CID/DCDC/Pages/COVID-19/SNFsCOVID_19.aspx', new=2)

True

### ENTER FILE DATES BEFORE RUNNING CELLS

In [169]:
##############################
###SET DATES BEFORE RUNNING###
##############################
#data_date = 'MM/DD/YYYY'
data_date ='09/21/2021'

#file_date = 'MMDDYY'
file_date = data_date.replace('2021', '21').replace('/','')
print(file_date)

092121


### FILE PATHS AND AGOL IDS

In [170]:
#specify file paths
#paths in
path_in = '../covid_data/snfs/raw/raw_snf_covid_{}.csv'.format(file_date)
path_snf = './data/snf_basic_info.csv'

#paths out
path_out = '../covid_data/snfs/snf_covid_{}.csv'.format(file_date)
path_out_active = '../covid_data/snfs/snf_covid_active_{}.csv'.format(file_date)
db_out = 'C:/Users/jesse/Dropbox/Mapping-Vulearable-Pop-Tasks/SD-County-Data/CA-Nursing-Facilities-Data/snf_covid_{}.csv'.format(file_date)

#AGOL IDs

# Cumulative Cases/Deaths dashboard (4 maps)
snf_layer1 = "55b29be4e904457f994713697f8f073b" # feature layer
res1 = "16dce30762734f579f7849bdafc8c019" # map
res2 = "a8319de392f64e4c836d42190ed10205" # map
hcw1 = "11543f486f414409b3106b98eb3e5fe4" # map
hcw2 = "d2cbaba1d61e43dab61aeb2fb2bd26de" # map
snf_dashboard1 = 'https://arcg.is/0bjCe1' # url to dashboard

# Active Cases dashboard (2 maps)
layer_id = '93dd409735d149b7b8ad484a78c23c9b' # feature layer
active_res_id = '27c0923fe74c4b7494d85e9e56b239f2' # map
active_hcw_id = '0286554fbd184deb99f752dc686b0705' # map
snf_dashboard2 = 'https://arcg.is/1zDKnz' # url to dashboard

# DATA PROCESSING

In [172]:
###################################################
###PREP WEBSITE DATA FOR MERGE WITH SPATIAL DATA###
###################################################

#read csv
df = pd.read_csv(path_in)

# replacing blank spaces with '_' in column names 
df.columns =[column.replace(" ", "_") for column in df.columns]
  
#delete unnecessary columns from df
del df['COUNTY']
del df['County_(copy)']
del df['FACILITY_NAME']
# del df['Measure_Names']
del df['NEW_CONFIRMED_POSITIVE_HCW___']
del df['NEW_CONFIRMED_POSITIVE_RESIDENTS_']
# del df['Measure_Values']
del df['AVAILABLE_BEDS_CAPABLE_OF_ISOLATION']
del df['AVAILABLE_BEDS']

#rename columns to match feature layer
df = df.rename(columns = {'Facility_Id':'FACID','COVID-RELATED_HCW_DEATHS_':'HCW_DEATHS_LABEL', 
                          'COVID-RELATED_RESIDENT_DEATHS':'RES_DEATHS_LABEL', 
                          'CUMULATIVE_POSITIVE_HCW_':'HCW_CUM_LABEL', 
                          'CUMULATIVE_POSITIVE_RESIDENTS_':'RES_CUM_LABEL'})

#duplicate columns 
#3 columns for each attribute: 
# label (includes <11), points (<11 as 1 for symbology), stats (<11 as 0 for calcs)
df['HCW_DEATHS_PTS'] = df['HCW_DEATHS_LABEL']
df['HCW_DEATHS_STATS'] = df['HCW_DEATHS_LABEL']

df['RES_DEATHS_PTS'] = df['RES_DEATHS_LABEL']
df['RES_DEATHS_STATS'] = df['RES_DEATHS_LABEL']

df['HCW_CUM_PTS'] = df['HCW_CUM_LABEL']
df['HCW_CUM_STATS'] = df['HCW_CUM_LABEL']

df['RES_CUM_PTS'] = df['RES_CUM_LABEL']
df['RES_CUM_STATS'] = df['RES_CUM_LABEL']

df['DATA_DATE'] = data_date

#format duplicated rows according to rules
#3 columns for each attribute: 
# label (includes <11), points (<11 as 1 for symbology), stats (<11 as 0 for calcs)
for i, row in df.iterrows():
    if(df['HCW_DEATHS_PTS'][i] == '<11'):
        df['HCW_DEATHS_PTS'][i] = 1
    if(df['HCW_DEATHS_STATS'][i] == '<11'):
        df['HCW_DEATHS_STATS'][i] = 0   
    if(df['RES_DEATHS_PTS'][i] == '<11'):
        df['RES_DEATHS_PTS'][i] = 1
    if(df['RES_DEATHS_STATS'][i] == '<11'):
        df['RES_DEATHS_STATS'][i] = 0 
    if(df['HCW_CUM_PTS'][i] == '<11'):
        df['HCW_CUM_PTS'][i] = 1
    if(df['HCW_CUM_STATS'][i] == '<11'):
        df['HCW_CUM_STATS'][i] = 0   
    if(df['RES_CUM_PTS'][i] == '<11'):
        df['RES_CUM_PTS'][i] = 1
    if(df['RES_CUM_STATS'][i] == '<11'):
        df['RES_CUM_STATS'][i] = 0 

#cast columns from string to int for feature layer
df['HCW_DEATHS_PTS'] = df['HCW_DEATHS_PTS'].astype(int)
df['HCW_DEATHS_STATS'] = df['HCW_DEATHS_STATS'].astype(int)
df['RES_DEATHS_PTS'] = df['RES_DEATHS_PTS'].astype(int)
df['RES_DEATHS_STATS'] = df['RES_DEATHS_STATS'].astype(int)
df['HCW_CUM_PTS'] = df['HCW_CUM_PTS'].astype(int)
df['HCW_CUM_STATS'] = df['HCW_CUM_STATS'].astype(int)
df['RES_CUM_PTS'] = df['RES_CUM_PTS'].astype(int)
df['RES_CUM_STATS'] = df['RES_CUM_STATS'].astype(int)

print(len(df))
df.head()

86


,FACILITY_ID,HCW_DEATHS_LABEL,RES_DEATHS_LABEL,HCW_CUM_LABEL,RES_CUM_LABEL,CURRENT_ACTIVE_CASES_RESIDENTS_,CURRENT_ACTIVE_HCW,HCW_DEATHS_PTS,HCW_DEATHS_STATS,RES_DEATHS_PTS,RES_DEATHS_STATS,HCW_CUM_PTS,HCW_CUM_STATS,RES_CUM_PTS,RES_CUM_STATS,DATA_DATE
0,80000056,0,12,56,63,0,0,0,0,12,12,56,56,63,63,09/21/2021
1,90000102,0,<11,28,27,0,0,0,0,1,0,28,28,27,27,09/21/2021
2,80000070,0,<11,63,55,0,0,0,0,1,0,63,63,55,55,09/21/2021
3,90000029,0,<11,21,36,<11,<11,0,0,1,0,21,21,36,36,09/21/2021
4,80000077,0,<11,58,68,0,<11,0,0,1,0,58,58,68,68,09/21/2021


In [173]:
###########################################
###MERGE WEBSITE & SPATIAL (COORDS) DATA###
###########################################

#read SNF file (addresses and coordinates)
snf = pd.read_csv(path_snf)

#merge website and spatial data
df = df.rename(columns = {'FACILITY_ID':'FACID'})
snf = snf.merge(df, on = 'FACID')

#########################################
###DETERMINE COUNTY TOTALS AND ADD ROW###
#########################################

#get county wide totals
hcw_deaths_sum = 0
res_deaths_sum = 0
hcw_cum_sum = 0
res_cum_sum = 0

for i,row in snf.iterrows():
    hcw_deaths_sum = hcw_deaths_sum + snf['HCW_DEATHS_STATS'][i]
    res_deaths_sum = res_deaths_sum + snf['RES_DEATHS_STATS'][i]
    hcw_cum_sum = hcw_cum_sum + snf['HCW_CUM_STATS'][i]
    res_cum_sum = res_cum_sum + snf['RES_CUM_STATS'][i]

print(hcw_deaths_sum, res_deaths_sum, hcw_cum_sum, res_cum_sum)
snf.head(2)

0 288 4059 4671


,Longitude,Latitude,COUNTY,FACID,FACILITY_NAME,SRA,ADDRESS,HCW_DEATHS_LABEL,RES_DEATHS_LABEL,HCW_CUM_LABEL,RES_CUM_LABEL,CURRENT_ACTIVE_CASES_RESIDENTS_,CURRENT_ACTIVE_HCW,HCW_DEATHS_PTS,HCW_DEATHS_STATS,RES_DEATHS_PTS,RES_DEATHS_STATS,HCW_CUM_PTS,HCW_CUM_STATS,RES_CUM_PTS,RES_CUM_STATS,DATA_DATE
0,-117.151726,32.800797,SAN DIEGO,80000056,ABBY GARDENS HEALTHCARE CENTER,KEARNY MESA,"8060 Frost St, San Diego, CA 92123",0,12,56,63,0,0,0,0,12,12,56,56,63,63,09/21/2021
1,-116.995140,32.739490,SAN DIEGO,90000102,AMAYA SPRINGS HEALTH CARE CENTER,SPRING VALLEY,"8625 Lamar St, Spring Valley, CA 91977",0,<11,28,27,0,0,0,0,1,0,28,28,27,27,09/21/2021


In [174]:
#######################################
# CREATE NEW DF WITH ACTIVE CASE INFO #
#######################################
# must create active df before deleting active case columns from the snf df

# columns for the 'Active Cases' feature layer (2 maps dashboard)
active = snf[['FACILITY_NAME', 'SRA', 'ADDRESS', 'FACID', 'DATA_DATE', 'HCW_DEATHS_LABEL', 
              'HCW_CUM_LABEL', 'CURRENT_ACTIVE_HCW', 'RES_DEATHS_LABEL', 'RES_CUM_LABEL', 
              'CURRENT_ACTIVE_CASES_RESIDENTS_', 'Longitude', 'Latitude']]

# rename columns to match schema
active = active.rename(columns = {'FACILITY_NAME':'Facility', 'ADDRESS':'Address', 'DATA_DATE':'Date', 
                                  'HCW_DEATHS_LABEL':'HCW Accum Deaths', 'HCW_CUM_LABEL':'HCW Accum Cases', 
                                  'CURRENT_ACTIVE_HCW':'HCW Active Cases', 
                                  'RES_DEATHS_LABEL':'Res Accum Deaths', 'RES_CUM_LABEL':'Res Accum Cases', 
                                  'CURRENT_ACTIVE_CASES_RESIDENTS_':'Res Active Cases'})

active.head(2)

,Facility,SRA,Address,FACID,Date,HCW Accum Deaths,HCW Accum Cases,HCW Active Cases,Res Accum Deaths,Res Accum Cases,Res Active Cases,Longitude,Latitude
0,ABBY GARDENS HEALTHCARE CENTER,KEARNY MESA,"8060 Frost St, San Diego, CA 92123",80000056,09/21/2021,0,56,0,12,63,0,-117.151726,32.800797
1,AMAYA SPRINGS HEALTH CARE CENTER,SPRING VALLEY,"8625 Lamar St, Spring Valley, CA 91977",90000102,09/21/2021,0,28,0,<11,27,0,-116.995140,32.739490


In [175]:
#########################################
# Format and save orginal SNF dataframe #
#########################################
# finish and save DF for the 'Cumulative Cases/Deaths' feature layer (4 maps dashboard)

# del active columns for snf df
del snf['CURRENT_ACTIVE_HCW']
del snf['CURRENT_ACTIVE_CASES_RESIDENTS_']

#create new row for county wide totals - snf df
new_row = {'Longitude':-118 , 'Latitude':33 , 'COUNTY': 'SAN DIEGO', 'FACID':999, 'FACILITY_NAME':' ALL SKILLED NURSING FACILITIES', 'SRA':'ALL SRAs', 'ADDRESS':'Totals do not include suppressed data for facilities with <11 confirmed cases or related deaths.', 'DATA_DATE':data_date, 'HCW_DEATHS_LABEL':hcw_deaths_sum, 'RES_DEATHS_LABEL':res_deaths_sum, 'HCW_CUM_LABEL':hcw_cum_sum, 'RES_CUM_LABEL':res_cum_sum, 'HCW_DEATHS_STATS':hcw_deaths_sum, 'RES_DEATHS_STATS':res_deaths_sum, 'HCW_CUM_STATS':hcw_cum_sum, 'RES_CUM_STATS':res_cum_sum, 'HCW_DEATHS_PTS':0, 'RES_DEATHS_PTS':0, 'HCW_CUM_PTS':0, 'RES_CUM_PTS':0}

#append row to dataframe
snf = snf.append(new_row, ignore_index=True)

#save as csv file for feature layer overwrite
snf.to_csv(path_out, index = False)

print(len(snf))
snf.tail(2)

87


,Longitude,Latitude,COUNTY,FACID,FACILITY_NAME,SRA,ADDRESS,HCW_DEATHS_LABEL,RES_DEATHS_LABEL,HCW_CUM_LABEL,RES_CUM_LABEL,HCW_DEATHS_PTS,HCW_DEATHS_STATS,RES_DEATHS_PTS,RES_DEATHS_STATS,HCW_CUM_PTS,HCW_CUM_STATS,RES_CUM_PTS,RES_CUM_STATS,DATA_DATE
85,-117.098743,32.66279,SAN DIEGO,90000042,WINDSOR GARDENS CONVALESCENT CENTER OF SAN DIEGO,NATIONAL CITY,"220 E 24th St, National City, CA 91950",<11,17,54,100,1,0,17,17,54,54,100,100,09/21/2021
86,-118.000000,33.00000,SAN DIEGO,999,ALL SKILLED NURSING FACILITIES,ALL SRAs,Totals do not include suppressed data for faci...,0,288,4059,4671,0,0,0,288,0,4059,0,4671,09/21/2021


In [176]:
###########################
# Format Active dataframe #
###########################

# add columns for point symbology and stats calcs for suppressed data to active df
active['HCW Death Points'] = active['HCW Accum Deaths']
active['HCW Death Stats'] = active['HCW Accum Deaths']
active['HCW Accum Points'] = active['HCW Accum Cases']
active['HCW Accum Stats'] = active['HCW Accum Cases']
active['HCW Active Points'] = active['HCW Active Cases']
active['HCW Active Stats'] = active['HCW Active Cases']

active['Res Death Points'] = active['Res Accum Deaths']
active['Res Death Stats'] = active['Res Accum Deaths']
active['Res Accum Points'] = active['Res Accum Cases']
active['Res Accum Stats'] = active['Res Accum Cases']
active['Res Active Points'] = active['Res Active Cases']
active['Res Active Stats'] = active['Res Active Cases']

# for points, change <11 to 1
# for stats, change <11 to 0
for i, row in active.iterrows():
    if active['HCW Death Points'][i] == '<11':
        active['HCW Death Points'][i] = 1
    if active['HCW Death Stats'][i] == '<11':
        active['HCW Death Stats'][i] = 0
    if active['HCW Accum Points'][i] == '<11':
        active['HCW Accum Points'][i] = 1
    if active['HCW Accum Stats'][i] == '<11':
        active['HCW Accum Stats'][i] = 0
    if active['HCW Active Points'][i] == '<11':
        active['HCW Active Points'][i] = 1
    if active['HCW Active Stats'][i] == '<11':
        active['HCW Active Stats'][i] = 0
        
    if active['Res Death Points'][i] == '<11':
        active['Res Death Points'][i] = 1
    if active['Res Death Stats'][i] == '<11':
        active['Res Death Stats'][i] = 0
    if active['Res Accum Points'][i] == '<11':
        active['Res Accum Points'][i] = 1
    if active['Res Accum Stats'][i] == '<11':
        active['Res Accum Stats'][i] = 0
    if active['Res Active Points'][i] == '<11':
        active['Res Active Points'][i] = 1
    if active['Res Active Stats'][i] == '<11':
        active['Res Active Stats'][i] = 0
        
# change points, stats columns to int type
active['HCW Death Points'] = active['HCW Death Points'].astype(int)
active['HCW Death Stats'] = active['HCW Death Stats'].astype(int)
active['HCW Accum Points'] = active['HCW Accum Points'].astype(int)
active['HCW Accum Stats'] = active['HCW Accum Stats'].astype(int)
active['HCW Active Points'] = active['HCW Active Points'].astype(int)
active['HCW Active Stats'] = active['HCW Active Stats'].astype(int)

active['Res Death Points'] = active['Res Death Points'].astype(int)
active['Res Death Stats'] = active['Res Death Stats'].astype(int)
active['Res Accum Points'] = active['Res Accum Points'].astype(int)
active['Res Accum Stats'] = active['Res Accum Stats'].astype(int)
active['Res Active Points'] = active['Res Active Points'].astype(int)
active['Res Active Stats'] = active['Res Active Stats'].astype(int)

print(len(active))
active.head(2)

86


,Facility,SRA,Address,FACID,Date,HCW Accum Deaths,HCW Accum Cases,HCW Active Cases,Res Accum Deaths,Res Accum Cases,Res Active Cases,Longitude,Latitude,HCW Death Points,HCW Death Stats,HCW Accum Points,HCW Accum Stats,HCW Active Points,HCW Active Stats,Res Death Points,Res Death Stats,Res Accum Points,Res Accum Stats,Res Active Points,Res Active Stats
0,ABBY GARDENS HEALTHCARE CENTER,KEARNY MESA,"8060 Frost St, San Diego, CA 92123",80000056,09/21/2021,0,56,0,12,63,0,-117.151726,32.800797,0,0,56,56,0,0,12,12,63,63,0,0
1,AMAYA SPRINGS HEALTH CARE CENTER,SPRING VALLEY,"8625 Lamar St, Spring Valley, CA 91977",90000102,09/21/2021,0,28,0,<11,27,0,-116.995140,32.739490,0,0,28,28,0,0,1,0,27,27,0,0


In [177]:
###################################
# Append summary row for all SRAs #
###################################

#create new row for county wide totals - active df
new_row = {'Facility':'All Skilled Nursing Facilities', 
           'SRA':'All SRAs', 
           'Address':'*Totals do not include suppressed data for facilities with <11 related deaths, accumulated cases, or active cases.', 
           'FACID': 999, 
           'Date': data_date, 
           'HCW Accum Deaths': str(active['HCW Death Stats'].sum()) + '*',
           'HCW Accum Cases': str(active['HCW Accum Stats'].sum()) + '*',
           'HCW Active Cases': str(active['HCW Active Stats'].sum()) + '*',
           'Res Accum Deaths': str(active['Res Death Stats'].sum()) + '*',
           'Res Accum Cases': str(active['Res Accum Stats'].sum()) + '*',
           'Res Active Cases': str(active['Res Active Stats'].sum()) + '*',
           'Latitude': 33,
           'Longitude': -118,
           'HCW Death Points': 0,
           'HCW Death Stats': active['HCW Death Stats'].sum(),
           'HCW Accum Points': 0,
           'HCW Accum Stats': active['HCW Accum Stats'].sum(),
           'HCW Active Points': 0,
           'HCW Active Stats': active['HCW Active Stats'].sum(),
           'Res Death Points': 0,
           'Res Death Stats': active['Res Death Stats'].sum(),
           'Res Accum Points': 0,
           'Res Accum Stats': active['Res Accum Stats'].sum(),
           'Res Active Points': 0,
           'Res Active Stats': active['Res Active Stats'].sum()
          }

#append row to dataframe
active = active.append(new_row, ignore_index=True)

#########################
# Save Active dataframe #
#########################

#save as csv file for feature layer overwrite
active.to_csv(path_out_active, index = False)

print(len(active))
active.tail(2)

87


,Facility,SRA,Address,FACID,Date,HCW Accum Deaths,HCW Accum Cases,HCW Active Cases,Res Accum Deaths,Res Accum Cases,Res Active Cases,Longitude,Latitude,HCW Death Points,HCW Death Stats,HCW Accum Points,HCW Accum Stats,HCW Active Points,HCW Active Stats,Res Death Points,Res Death Stats,Res Accum Points,Res Accum Stats,Res Active Points,Res Active Stats
85,WINDSOR GARDENS CONVALESCENT CENTER OF SAN DIEGO,NATIONAL CITY,"220 E 24th St, National City, CA 91950",90000042,09/21/2021,<11,54,0,17,100,0,-117.098743,32.66279,1,0,54,54,0,0,17,17,100,100,0,0
86,All Skilled Nursing Facilities,All SRAs,*Totals do not include suppressed data for fac...,999,09/21/2021,0*,4059*,0*,288*,4671*,13*,-118.000000,33.00000,0,0,0,4059,0,0,0,288,0,4671,0,13


In [178]:
#####################################
# Subset Active df for Dropbox save #
#####################################

active2 = active.iloc[:,0:13]
active2.to_csv(db_out, index = False)

print(len(active2))
active2.tail(2)

87


,Facility,SRA,Address,FACID,Date,HCW Accum Deaths,HCW Accum Cases,HCW Active Cases,Res Accum Deaths,Res Accum Cases,Res Active Cases,Longitude,Latitude
85,WINDSOR GARDENS CONVALESCENT CENTER OF SAN DIEGO,NATIONAL CITY,"220 E 24th St, National City, CA 91950",90000042,09/21/2021,<11,54,0,17,100,0,-117.098743,32.66279
86,All Skilled Nursing Facilities,All SRAs,*Totals do not include suppressed data for fac...,999,09/21/2021,0*,4059*,0*,288*,4671*,13*,-118.000000,33.00000


# CONNECT TO AGOL

In [179]:
#connect to ArcGIS account
gis = GIS("pro")

### FUNCTIONS FOR MAP MODIFICATIONS

Code Reference: https://community.esri.com/groups/arcgis-python-api/blog/2019/04/09/updating-layer-symbology-with-the-arcgis-api-for-python

In [180]:
def get_map (map_id):
    '''
    GET MAP DATA FOR SYMBOLOGY CHANGES
    '''   
    m = gis.content.get(map_id)
    data = m.get_data()    
    print(m)    
    return data

In [181]:
def update_map (map_id, data):
    '''
    UPDATE MAP TO SAVE CHANGES
    '''

    m = gis.content.get(map_id)
    
    # Set the item_properties to include the desired update
    properties = {"text": json.dumps(data)}

    # 'Commit' the updates to the Item
    update = m.update(item_properties=properties)

    # Print item_data to see that changes are reflected
    #new_data = m.get_data()
    #print("***********************NEW DEFINITION**********************")
    #print(json.dumps(new_data, indent=4, sort_keys=True))
    
    return update

# CUMULATIVE CASES AND DEATHS DASHBOARD

### GET MAX VALUES FOR MAP SYMBOL UPDATES

In [182]:
# get max deaths and max cumulative cases for point symbology corrections

# CASES
max_res_cum = snf['RES_CUM_STATS'].max()
print('Max Res Cum: {}'.format(max_res_cum))

max_hcw_cum = snf['HCW_CUM_STATS'].max()
print('Max HCW Cum: {}'.format(max_hcw_cum))

# Determine if HCW max or resident max is larger, use for both maps
max_cum = 0
if(max_res_cum > max_hcw_cum):
    max_cum = max_res_cum
else:
    max_cum = max_hcw_cum
print('Max Cum: {}'.format(max_cum))

# DEATHS
max_res_deaths = snf['RES_DEATHS_STATS'].max()
print('Max Res Deaths: {}'.format(max_res_deaths))

max_hcw_deaths = snf['HCW_DEATHS_STATS'].max()
print('Max HCW Deaths: {}'.format(max_hcw_deaths))

# Determine if HCW max or resident max is larger, use for both maps
max_deaths = 0
if(max_res_deaths > max_hcw_deaths):
    max_deaths = max_res_deaths
else:
    max_deaths = max_hcw_deaths
print('Max Deaths: {}'.format(max_deaths))

Max Res Cum: 4671
Max HCW Cum: 4059
Max Cum: 4671
Max Res Deaths: 288
Max HCW Deaths: 0
Max Deaths: 288


### OVERWRITE FEATURE LAYER

In [183]:
#############################
###OVERWRITE FEATURE LAYER###
#############################

#get feature layer containing updated data for maps associated with the COVID-19 dashboards
layer = gis.content.get(snf_layer1)
layer

layer_collection = FeatureLayerCollection.fromitem(layer)

#call the overwrite() method which can be accessed using the manager property
layer_collection.manager.overwrite(path_out)


{'success': True}

### UPDATE MAPS (x4)

In [184]:
#residents - cumulative cases
res1_data = get_map(res1)

#set max value for graduated points symbols
res1_data['operationalLayers'][4]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_cum.item()
res1_data['operationalLayers'][4]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_cum.item()

#update map to save changes
res1_update = update_map(res1, res1_data)
res1_update

<Item title:"COVID-19 and San Diego County SNF Residents" type:Web Map owner:jembury_SDSUGeo>


True

In [185]:
#residents - deaths
res2_data = get_map(res2)

#set max value for graduated points symbols
res2_data['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_deaths.item()
res2_data['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_deaths.item()

#update map to save changes
res2_update = update_map(res2, res2_data)
res2_update

<Item title:"COVID-19 and San Diego County SNF Resident Deaths" type:Web Map owner:jembury_SDSUGeo>


True

In [186]:
#healthcare workers - cumulative cases
hcw1_data = get_map(hcw1)

#set max value for graduated points symbols
hcw1_data['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_cum.item()
hcw1_data['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_cum.item()

#update map to save changes
hcw1_update = update_map(hcw1, hcw1_data)
hcw1_update

<Item title:"COVID-19 and San Diego County SNF Healthcare Workers" type:Web Map owner:jembury_SDSUGeo>


True

In [187]:
#healthcare workers - deaths
hcw2_data = get_map(hcw2)

#set max value for graduated points symbols
hcw2_data['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_deaths.item()
hcw2_data['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_deaths.item()

#update map to save changes
hcw2_update = update_map(hcw2, hcw2_data)
hcw2_update

<Item title:"COVID-19 and San Diego County SNF Healthcare Worker Deaths" type:Web Map owner:jembury_SDSUGeo>


True

# ACTIVE CASES DASHBOARD

### GET MAX VALUES FOR MAP SYMBOL UPDATES

In [188]:
#######################################
# get max values for symbology update #
#######################################
active = active.query("FACID != 999")

max_active_hcw = active['HCW Active Stats'].max()
max_active_res = active['Res Active Stats'].max()

max_accum_hcw = active['HCW Accum Stats'].max()
max_accum_res = active['Res Accum Stats'].max()

print('Max Active: Res {}/ HCW {}; Max Accum.: Res {}/ HCW {}'.format(max_active_res, max_active_hcw, max_accum_res, max_accum_hcw))

max_active = 0
if(max_active_res > max_active_hcw):
    max_active = max_active_res
else:
    max_active = max_active_hcw
    
max_accum = 0
if(max_accum_res > max_accum_hcw):
    max_accum = max_accum_res
else:
    max_accum = max_accum_hcw
    
print('Max Active: {}; Max Accum: {}'.format(max_active, max_accum))

Max Active: Res 13/ HCW 0; Max Accum.: Res 286/ HCW 145
Max Active: 13; Max Accum: 286


### OVERWRITE FEATURE LAYER

In [189]:
###########################
# Overwrite Feature Layer #
###########################

#get feature layer containing updated data for maps associated with the COVID-19 dashboards
layer = gis.content.get(layer_id)
layer

layer_collection = FeatureLayerCollection.fromitem(layer)

#call the overwrite() method which can be accessed using the manager property
layer_collection.manager.overwrite(path_out_active)

{'success': True}

### UPDATE MAPS (x2)

In [190]:
#########################
# Update Active Res Map #
#########################

#get data
active_res_data = get_map(active_res_id)

#set max value for graduated points symbols
active_res_data['operationalLayers'][4]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_active.item()
active_res_data['operationalLayers'][4]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_active.item()

active_res_data['operationalLayers'][3]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_accum.item()
active_res_data['operationalLayers'][3]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_accum.item()

#update map to save changes
active_res_update = update_map(active_res_id, active_res_data)
active_res_update

<Item title:"Active COVID-19 in SD County SNFs - Residents" type:Web Map owner:jembury_SDSUGeo>


True

In [191]:
#########################
# Update Active HCW Map #
#########################

#get data
active_hcw_data = get_map(active_hcw_id)

#set max value for graduated points symbols
active_hcw_data['operationalLayers'][4]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_active.item()
active_hcw_data['operationalLayers'][4]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_active.item()

active_hcw_data['operationalLayers'][3]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_accum.item()
active_hcw_data['operationalLayers'][3]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_accum.item()

#update map to save changes
active_hcw_update = update_map(active_hcw_id, active_hcw_data)
active_hcw_update

<Item title:"Active COVID-19 in SD County SNFs - HCW" type:Web Map owner:jembury_SDSUGeo>


True

# VERIFY COVID-19 SNF DASHBOARD

In [192]:
#Open COVID-19 SNF dashboards in browser
webbrowser.open(snf_dashboard1, new=2)
webbrowser.open(snf_dashboard2, new=2)

True